In [8]:
from datetime import date
import requests
import re
from pprint import pprint
import os
import csv

In [9]:
class Persona:
    def __init__(self, nombre = "Sin nombre", apellido = "Sin apellido", fecha_nacimiento = (1900, 1, 1)):
        self.nombre = nombre
        self.apellido = apellido
        self.fecha_nacimiento = date(*fecha_nacimiento)

    def saludos(self):
        print("Saludos, soy", self.nombre, self.apellido)

    def calcula_edad(self):
        today = date.today()
        edad = int((today - self.fecha_nacimiento).days / 365)
        return edad

In [10]:
class Estudiante(Persona):
    def __init__(self, nombre = "Sin nombre", apellido = "Sin apellido", fecha_nacimiento = (1900, 1, 1),
                 semestre:int = 1):
        super().__init__(nombre, apellido, fecha_nacimiento)
        self.semestre = semestre
        self.materias = []
        self.calificaciones = {}
        self.__meses__ = {1:"Enero",2:"Febrero",3:"Marzo",4:"Abril",5:"Mayo",\
                          6:"Junio",7:"Julio",8:"Agosto",9:"Septiembre",\
                            10:"Octubre",11:"Noviembre",12:"Diciembre"}
        #Add an promedio attribute
        self.promedio = 0

    def setMatricula(self):
        if self.fecha_nacimiento.year == 1900:
            print("Error: Fecha de nacimiento inválida")
        else:
            self.matricula = str(self.fecha_nacimiento.year) + self.nombre[:2] + self.apellido[:2]
        return

    def getMatricula(self):
        return self.matricula

    def setFechaIngreso(self,fecha:tuple):
        self.fecha_ingreso = date(*fecha)
        return

    def setSemestre(self,semestre:int=1):
        self.semestre = semestre
        return

    def getFechaIngreso(self):
        anio = self.fecha_ingreso.year
        mes = self.fecha_ingreso.month
        dia = self.fecha_ingreso.day
        print(f"El estudiante {self.nombre} {self.apellido} ingresó el {dia} de {self.__meses__[mes]} de {anio}")
        return

    def __str__(self) -> str:
        return f"{self.nombre} {self.apellido} del semestre {self.semestre} obtuvo un promedio de {self.promedio}"


In [42]:
class Registro():
    def __init__(self,*args:Estudiante,**kwargs):
        # print('args =',args, 'kwargs =',kwargs, *kwargs.get("estudiante"))
        if kwargs.get("estudiante") != None:
            self.estudiante = Estudiante(*kwargs.get("estudiante"))
        else:
            self.estudiante = args[0] # objeto de tipo Estudiante

    #changed to accept a dict
    def setCalificaciones(self,**kwargs):
        if len(kwargs) ==0:
            print("Error: Ningún registro de calificaciones proporcionado")
        else:
            if kwargs.get('registros') != None:
                if len(kwargs.get('registros')) == 0:
                    print("Error: Ningún registro de calificaciones proporcionado")
                else:
                    registros = kwargs.get('registros')
                    for r in registros:
                        self.estudiante.calificaciones[r[0]] = r[1]
                    self.estudiante.materias.append(r[0])
                    self.getPromedio()
            elif kwargs.get('materias') != None:
                materias = kwargs.get('materias')
                for materia in materias.keys():
                    self.estudiante.calificaciones[materia] = materias[materia]
                    self.estudiante.materias.append(materia)
                self.getPromedio()
        return

    def printCalificaciones(self):
        if len(self.estudiante.calificaciones) == 0:
            print("Error: no se han capturado calificaciones")
        else:
            print(f"{self.estudiante.nombre} {self.estudiante.apellido} \nSemestre: {self.estudiante.semestre}")
            for m in self.estudiante.calificaciones.keys():
                print(f"{m}: {self.estudiante.calificaciones[m]}")
            print(f"Promedio: {self.estudiante.promedio}")
        return

    #GET PROMEDIO
    def getPromedio(self):
        if len(self.estudiante.calificaciones) == 0:
            print("Error: no se han capturado calificaciones")
        else:
            avg = 0
            for m in self.estudiante.calificaciones.keys():
                avg += float(self.estudiante.calificaciones[m])
            self.estudiante.promedio = round(avg/len(self.estudiante.calificaciones), 2)
        return self.estudiante.promedio

    def __str__(self):
        return f"Registro: {self.estudiante}"


In [70]:
class Sistema():
  def __init__(self,*args,**kwargs):
        # print('args =',args, 'kwargs =',kwargs, *kwargs.get("estudiante"))
        self.registros = []
        self.folder_path = "registros_info"
        self.semester_average = {1:[],
                                 2:[],
                                 3:[],
                                 4:[],
                                 5:[],
                                 6:[],
                                 7:[],
                                 8:[],
                                 9:[]}
        self.materias_avg = {}
        self.archivos_url = 'https://raw.githubusercontent.com/DCDPUAEM/DCDP/main/01%20Programaci%C3%B3n%20en%20Python/notebooks/sample_data'
        self.folder_url = "https://github.com/DCDPUAEM/DCDP/tree/main/01%20Programaci%C3%B3n%20en%20Python/notebooks/sample_data"
        if kwargs.get("folder_path") != None:
            self.folder_path = kwargs.get("folder_path")
        self.get_files()
        self.getRegistros()
        self.setSemestre_and_Scores()
        self.get_Semester_Average()
        self.get_materias_avg()


  def get_files(self, **kwargs):
    if len(kwargs) !=0:
        print("Error: Ningún dato proporcionado")
    else:
        response = requests.get(self.folder_url).json()['payload']['tree']['items']

        file_list_json = response


        file_list_filtered = [item for item in file_list_json if (re.match(r'^c.*\d+\.csv$', item['name']) or re.match(r'^estudiantes.csv$', item['name']))]

        if not os.path.exists(self.folder_path):
          os.makedirs(self.folder_path)

          for file in file_list_filtered:
            response = requests.get(f'{self.archivos_url}/{file["name"]}')
            file_path = os.path.join(self.folder_path, file['name'])
            if response.status_code == 200 and not os.path.exists(file_path):
              file_path_url = f"{self.archivos_url}/{file['name']}"
              with open(file_path, 'wb') as f:
                f.write(requests.get(file_path_url).content)

  def getRegistros(self):
    estudiantes_path = os.path.join(self.folder_path, 'estudiantes.csv')
    with open(estudiantes_path) as archivo:
      registros = csv.reader(archivo, delimiter=',',quotechar=',')
      for r in list(registros)[1:]:
        fecha_nac = (int(r[3][2:]),int(r[4]),int(r[5][:-2]))
        self.registros.append(Registro(estudiante=[r[1],r[2],fecha_nac]))

  def setSemestre_and_Scores(self):
    file_list = os.listdir(self.folder_path)
    filtered_files = [file_name for file_name in file_list if re.match(r'^c.*\d+\.csv$', file_name)]

    def extract_number(file_name):
      return int(re.search(r'\d+', file_name).group())

    for i, filtered_file in enumerate(filtered_files):
      materias = {}
      filtered_file_path = f"{self.folder_path}/{filtered_file}"
      with open(filtered_file_path, 'r') as archivo:
        registros_rows = csv.reader(archivo, delimiter=',',quotechar=',')
        header = next(registros_rows)
        semestre = header[2]
        self.registros[i].estudiante.semestre = semestre
        for row in registros_rows:
          materias[row[0]] = row[1]
        self.registros[i].setCalificaciones(materias = materias)

    filtered_files = sorted(filtered_files, key=extract_number)

  def get_Semester_Average(self):
    for registro in self.registros:
        semestre = int(registro.estudiante.semestre)
        self.semester_average[semestre].append( registro.estudiante.promedio)

    with open("semester_average.csv", 'w', newline='') as archivo:
        writer = csv.writer(archivo)
        writer.writerow(['Semester', 'Promedio'])  # Writing header

        for semester, averages in self.semester_average.items():
            semester_avg = sum(averages) / len(averages) if averages else 0
            writer.writerow([semester, semester_avg])

  def print_semester_average(self):
    print("Semester Average")
    with open('semester_average.csv', 'r') as archivo:
        semesters = csv.reader(archivo, delimiter=',', quotechar=',')
        next(semesters)  # Skip the header row
        for row in semesters:
            print(f"Semestre {row[0]} = {row[1]}")

  def get_materias_avg(self):
    for registro in self.registros:
        for materia, score in registro.estudiante.calificaciones.items():
            if materia not in self.materias_avg:
                self.materias_avg[materia] = []
            self.materias_avg[materia].append(float(score))
    with open("materias_average.csv", 'w', newline='') as archivo:
        writer = csv.writer(archivo)
        writer.writerow(['Materia', 'Promedio'])  # Writing header

        for materia, scores in self.materias_avg.items():
            materia_avg = sum(scores) / len(scores) if scores else 0
            writer.writerow([materia, materia_avg])


    """with open("materias_average.csv", 'w', newline='') as archivo:
        writer = csv.writer(archivo)
        writer.writerow(['Materia'])"""

  def print_materia_average(self):
    print("Materia Average")
    with open('materias_average.csv', 'r') as archivo:
        materias = csv.reader(archivo, delimiter=',', quotechar=',')
        next(materias)  # Skip the header row
        for row in materias:
            print(f"Materia {row[0]} = {row[1]}")
  def print_registros(self):
      for r in self.registros:
        print(r.estudiante)

In [71]:
sistema_registros = Sistema()
sistema_registros.print_semester_average()
sistema_registros.print_registros()
sistema_registros.print_materia_average()

Semester Average
Semestre 1 = 7.6
Semestre 2 = 7.666666666666667
Semestre 3 = 0
Semestre 4 = 7.133333333333333
Semestre 5 = 0
Semestre 6 = 0
Semestre 7 = 7.0
Semestre 8 = 0
Semestre 9 = 0
Juan Perez del semestre 4 obtuvo un promedio de 5.4
Ana Aranda del semestre 2 obtuvo un promedio de 6.4
Luis Rojas del semestre 2 obtuvo un promedio de 7.8
Diana Valenzo del semestre 2 obtuvo un promedio de 7.0
Lorena Diaz del semestre 2 obtuvo un promedio de 9.4
Jorge Diaz del semestre 2 obtuvo un promedio de 9.0
Gonzalo Vega del semestre 2 obtuvo un promedio de 6.4
Rodrigo Hernandez del semestre 1 obtuvo un promedio de 8.0
Carmela Hernandez del semestre 4 obtuvo un promedio de 6.6
Veronica Neri del semestre 7 obtuvo un promedio de 7.0
Antonio Gamboa del semestre 1 obtuvo un promedio de 7.2
Luis Gonzalez del semestre 4 obtuvo un promedio de 9.4
Materia Average
Materia inteligencia artificial = 7.333333333333333
Materia logica = 7.333333333333333
Materia programacion 3 = 6.666666666666667
Materia prob